#### Imports

In [1]:
import datetime
import numpy as np
import pandas as pd
import random
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for machine learning
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## for statistical tests
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm
 
## for explainer
from lime import lime_tabular
    
## plot style and warnings    
plt.style.use('ggplot')
from IPython.display import display # Allows the use of display() for DataFrames
import warnings
warnings.filterwarnings('ignore')

### Abertura da Base de Dados

In [2]:
df = pd.read_csv('atpplayers.csv', sep=',')

# **Data Cleaning**

### Informaçãoes sobre a base de dados

In [3]:
df.head(6)

,PlayerName,Born,Height,Hand,LinkPlayer,Tournament,Location,Date,Ground,Prize,GameRound,GameRank,Oponent,WL,Score
0,Novak Djokovic,"Belgrade, Serbia",188.0,"Right-Handed, Two-Handed Backhand",https://www.atptour.com/en/players/novak-djoko...,Dubai,"Dubai, U.A.E.",2022.02.21 - 2022.02.26,Hard,"$2,794,840",Round of 16,26,Karen Khachanov,W,63 76
1,Novak Djokovic,"Belgrade, Serbia",188.0,"Right-Handed, Two-Handed Backhand",https://www.atptour.com/en/players/novak-djoko...,Davis Cup Finals,"Madrid, Spain",2021.11.22 - 2021.12.05,Hard,NaN,Round Robin,51,Jan-Lennard Struff,W,62 64
2,Novak Djokovic,"Belgrade, Serbia",188.0,"Right-Handed, Two-Handed Backhand",https://www.atptour.com/en/players/novak-djoko...,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,"$27,200,000",Round of 128,145,Holger Rune,W,"61 67, 62 61"
3,Novak Djokovic,"Belgrade, Serbia",188.0,"Right-Handed, Two-Handed Backhand",https://www.atptour.com/en/players/novak-djoko...,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,"$27,200,000",Round of 64,121,Tallon Griekspoor,W,62 63 62
4,Novak Djokovic,"Belgrade, Serbia",188.0,"Right-Handed, Two-Handed Backhand",https://www.atptour.com/en/players/novak-djoko...,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,"$27,200,000",Semi-Finals,4,Alexander Zverev,W,46 62 64 46 62
5,Novak Djokovic,"Belgrade, Serbia",188.0,"Right-Handed, Two-Handed Backhand",https://www.atptour.com/en/players/novak-djoko...,Davis Cup Finals,"Madrid, Spain",2021.11.22 - 2021.12.05,Hard,NaN,Semi-Finals,30,Marin Cilic,W,64 62


In [4]:
df.shape

(1308835, 15)

In [5]:
df.columns

Index(['PlayerName', 'Born', 'Height', 'Hand', 'LinkPlayer', 'Tournament',
       'Location', 'Date', 'Ground', 'Prize', 'GameRound', 'GameRank',
       'Oponent', 'WL', 'Score'],
      dtype='object')

### Transformações à base de dados

Foram apagadas as variáveis que não vão ser utilizadas.

In [6]:
df = df.drop(['Hand', 'LinkPlayer'], axis=1)

In [7]:
df = df.drop(['Height'], axis=1)

In [8]:
df = df.drop(['Born'], axis=1)

In [9]:
df.isna().sum()

PlayerName        0
Tournament        0
Location          0
Date              0
Ground          191
Prize         19136
GameRound         0
GameRank      19236
Oponent           7
WL            19257
Score         19400
dtype: int64

In [10]:
df.dropna(subset=['Score'], inplace=True)

In [11]:
df.isna().sum()

PlayerName        0
Tournament        0
Location          0
Date              0
Ground          191
Prize         19109
GameRound         0
GameRank          0
Oponent           7
WL                0
Score             0
dtype: int64

In [12]:
df = df.drop(['Prize'], axis=1)

In [13]:
df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score
0,Novak Djokovic,Dubai,"Dubai, U.A.E.",2022.02.21 - 2022.02.26,Hard,Round of 16,26,Karen Khachanov,W,63 76
1,Novak Djokovic,Davis Cup Finals,"Madrid, Spain",2021.11.22 - 2021.12.05,Hard,Round Robin,51,Jan-Lennard Struff,W,62 64
2,Novak Djokovic,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,Round of 128,145,Holger Rune,W,"61 67, 62 61"
3,Novak Djokovic,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,Round of 64,121,Tallon Griekspoor,W,62 63 62
4,Novak Djokovic,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,Semi-Finals,4,Alexander Zverev,W,46 62 64 46 62
5,Novak Djokovic,Davis Cup Finals,"Madrid, Spain",2021.11.22 - 2021.12.05,Hard,Semi-Finals,30,Marin Cilic,W,64 62


Criação de uma base de dados espelhada onde o opponent vai para o playername e vice-versa.

In [14]:
df_mirror = df.copy()
df_mirror['Oponent'], df_mirror['PlayerName'] = df_mirror['PlayerName'], df_mirror['Oponent']

df_concat = pd.concat([df, df_mirror], axis=0, ignore_index=True)
counted = df_concat.groupby(['Date', 'GameRound', 'Oponent', 'PlayerName']).size().reset_index(name='count')

matching_rows = counted[counted['count'] > 1]
print(len(matching_rows))

unmatched_rows = df[~df.set_index(['Date', 'GameRound', 'Oponent', 'PlayerName']).index.isin(matching_rows.set_index(['Date', 'GameRound', 'Oponent', 'PlayerName']).index)]
unmatched_list = unmatched_rows.to_dict('records')

my_list = list(unmatched_list)

1211700


In [15]:
print(df.shape[0] - len(matching_rows))

77735


In [16]:
df_nao_espelhados=pd.DataFrame(my_list)
print(len(df_nao_espelhados))

76119


In [17]:
len(df_nao_espelhados[df_nao_espelhados['WL'] == 'W'])

54312

In [18]:
df_nao_espelhados = df_nao_espelhados[df_nao_espelhados['WL'] == 'W']
len(df_nao_espelhados)

54312

In [19]:
matched_df = pd.merge(df, matching_rows, on=['Date', 'GameRound', 'Oponent', 'PlayerName'], how='inner')
matched_df.shape

(1213316, 11)

In [20]:
matched_df = matched_df.drop('count', axis=1)
matched_df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score
0,Novak Djokovic,Dubai,"Dubai, U.A.E.",2022.02.21 - 2022.02.26,Hard,Round of 16,26,Karen Khachanov,W,63 76
1,Novak Djokovic,Davis Cup Finals,"Madrid, Spain",2021.11.22 - 2021.12.05,Hard,Round Robin,51,Jan-Lennard Struff,W,62 64
2,Novak Djokovic,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,Round of 128,145,Holger Rune,W,"61 67, 62 61"
3,Novak Djokovic,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,Round of 64,121,Tallon Griekspoor,W,62 63 62
4,Novak Djokovic,US Open,"New York, NY, U.S.A.",2021.08.30 - 2021.09.12,Hard,Semi-Finals,4,Alexander Zverev,W,46 62 64 46 62
5,Novak Djokovic,Davis Cup Finals,"Madrid, Spain",2021.11.22 - 2021.12.05,Hard,Semi-Finals,30,Marin Cilic,W,64 62


In [21]:
matched_df = matched_df[matched_df['WL'] == 'W']
len(matched_df)

606955

In [22]:
new_df = pd.concat([df_nao_espelhados, matched_df], axis=0, ignore_index=True)
len(new_df)

661267

In [23]:
new_df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score
0,Novak Djokovic,ATP Masters 1000 Canada,"Toronto, Canada",2016.07.25 - 2016.07.31,Hard,Quarter-Finals,8,Tomas Berdych,W,"76, 64"
1,Novak Djokovic,Roland Garros,"Paris, France",2016.05.23 - 2016.06.05,Clay,Quarter-Finals,8,Tomas Berdych,W,63 75 63
2,Novak Djokovic,ATP Masters 1000 Miami,"Miami, FL, U.S.A.",2016.03.21 - 2016.04.03,Hard,Quarter-Finals,7,Tomas Berdych,W,63 63
3,Novak Djokovic,ATP Finals,"London, Great Britain",2015.11.16 - 2015.11.22,Hard,Round Robin,6,Tomas Berdych,W,63 75
4,Novak Djokovic,Doha,"Doha, Qatar",2016.01.04 - 2016.01.10,Hard,Semi-Finals,6,Tomas Berdych,W,63 76
5,Novak Djokovic,ATP Masters 1000 Paris,"Paris, France",2015.11.02 - 2015.11.08,Hard,Quarter-Finals,5,Tomas Berdych,W,"76, 76"


In [24]:
new_df.loc[(new_df['PlayerName'] == "Boris Becker")]

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score
34917,Boris Becker,Australian Open,"Melbourne, Australia",1990.01.15 - 1990.01.28,Hard,Round of 16,25,Miloslav Mecir Sr.,W,46 67 64 61 61
34918,Boris Becker,Stuttgart-1,"Stuttgart, Germany",1990.02.19 - 1990.02.25,Carpet,Quarter-Finals,26,Miloslav Mecir Sr.,W,64 62
34919,Boris Becker,US Open,"New York, NY, U.S.A.",1989.08.28 - 1989.09.10,Hard,Round of 32,22,Miloslav Mecir Sr.,W,64 36 64 63
34920,Boris Becker,IND V GER 1RD,Germany,1989.01.30 - 1989.02.12,Carpet,Round Robin,860,Abdul Kahar Mim,W,60 61 61
34921,Boris Becker,IND V GER 1RD,Germany,1989.01.30 - 1989.02.12,Carpet,Round Robin,412,Tintus Wibowo,W,62 75
...,...,...,...,...,...,...,...,...,...,...
571114,Boris Becker,Wembley,"Wembley, England",1984.11.05 - 1984.11.11,Carpet,Round of 32,92,Emilio Sanchez,W,64 61
571115,Boris Becker,Vienna,"Vienna, Austria",1984.10.22 - 1984.10.28,Hard,Round of 32,145,Karel Novacek,W,64 67 61
571116,Boris Becker,Wimbledon,"London, England",1984.06.25,Grass,Round of 64,52,Nduka Odizor,W,63 64 42 (RET)
571117,Boris Becker,Hamburg,"Hamburg, Germany",1984.05.07,Clay,Round of 64,-,Sergio Casal,W,63 67 61


In [25]:
new_df.loc[(new_df['PlayerName'] == "Boris Becker") & (new_df["Oponent"]=="John McEnroe")] 

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score
570706,Boris Becker,ATP Masters 1000 Paris,"Paris, France",1992.11.02 - 1992.11.08,Carpet,Round of 32,22,John McEnroe,W,64 64
570743,Boris Becker,Rotterdam,"Rotterdam, Netherlands",1992.02.24 - 1992.03.01,Carpet,Semi-Finals,30,John McEnroe,W,62 76
570873,Boris Becker,Masters,"NY, U.S.A.",1989.11.27 - 1989.12.03,Carpet,Semi-Finals,4,John McEnroe,W,64 64
570884,Boris Becker,Paris,"Paris, France",1989.10.30 - 1989.11.05,Carpet,Semi-Finals,4,John McEnroe,W,76 36 63
570928,Boris Becker,Milan,"Milan, Italy",1989.02.13 - 1989.02.19,Carpet,Semi-Finals,9,John McEnroe,W,62 63
570949,Boris Becker,Indianapolis,"Indianapolis, IN, U.S.A.",1988.08.01 - 1988.08.07,Hard,Finals,17,John McEnroe,W,64 62
571005,Boris Becker,GER V USA WGPO,U.S.A.,1987.07.20 - 1987.07.26,Carpet,Round Robin,9,John McEnroe,W,46 15-138-1062 62
571057,Boris Becker,Stratton Mountain,"Stratton Mountain, VT, U.S.A.",1986.08.04 - 1986.08.10,Hard,Semi-Finals,7,John McEnroe,W,36 75 76


In [26]:
new_df['Score_Sets'] = new_df['Score'].apply(lambda x: x.replace(' ', '').replace(',', '').replace('-', ''))
new_df['Score_Sets'] = new_df['Score_Sets'].str.replace('(RET)', '').str.replace('(DEF)', '')
new_df['Score_Sets'] = new_df['Score_Sets'].str.replace(r'\(.*\)', '')
import re
new_df["Score_Sets"] = new_df["Score_Sets"].apply(lambda x: re.sub(r"(\d{2})", r"\1-", x)[:-1])

In [27]:
def count_sets(row):
    sets = row['Score_Sets']
    if '-' not in sets:
        return 1
    else:
        return sets.count('-') + 1

new_df['Sets'] = new_df.apply(count_sets, axis=1)

In [28]:
new_df['Sets'].value_counts()

2    435205
3    202389
1      9850
4      8683
5      4872
6       267
7         1
Name: Sets, dtype: int64

In [29]:
new_df.loc[(new_df['Sets'] == 7) | (new_df['Sets'] == 6)]

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,Score_Sets,Sets
433,Grigor Dimitrov,Roland Garros,"Paris, France",2018.05.28 - 2018.06.10,Clay,Round of 64,57,Jared Donaldson,W,"67, 64 46 64 10-8",67-64-46-64-10-,6
29712,Wesley Moodie,RSA v. GER EAGI 2nd RD,"Doornfontein, South",2005.02.28 - 2005.03.06,Hard,Round Robin,16,Tommy Haas,W,"67, 62 46 63 10-8",67-62-46-63-10-,6
33537,John James,Adelaide,"Adelaide, Australia",1968.11.25 - 1968.12.01,Grass,Round of 64,-,Jeff Simpson,W,46 75 61 8-1063,46-75-61-81-06-,6
35115,Leonardo Lavalle,GER V MEX 1RD,Mexico,1986.03.03 - 1986.03.16,Clay,Round Robin,50,Michael Westphal,W,8-1036 63 64 63,81-03-66-36-46-,6
35475,Jean-Noel Grinda,Wimbledon,"London, Great Britain",1954.06.21 - 1954.07.02,Grass,Round of 128,-,Guy Jackson,W,64 64 8-1036 75,64-64-81-03-67-,6
...,...,...,...,...,...,...,...,...,...,...,...,...
614892,Harold Solomon,Roland Garros,"Paris, France",1972.05.22 - 1972.06.04,Clay,Round of 16,-,Guillermo Vilas,W,36 8-1062 64 64,36-81-06-26-46-,6
614893,Harold Solomon,Roland Garros,"Paris, France",1972.05.22 - 1972.06.04,Clay,Round of 16,-,Guillermo Vilas,W,36 8-1062 64 64,36-81-06-26-46-,6
615121,Peter Doohan,Wimbledon,"London, England",1987.06.22 - 1987.07.05,Grass,Round of 32,166,Leif Shiras,W,67 46 63 64 12-10,67-46-63-64-12-10,6
630858,Andrey Golubev,KAZ vs. BEL WG 1st RD,Kazakhstan,2014.01.27 - 2014.02.02,Hard,Round Robin,113,David Goffin,W,"76, 36 46 62 12-10",76-36-46-62-12-10,6


In [30]:
new_df['Sets'] = new_df['Sets'].replace([6, 7], 5)

In [31]:
new_df['Sets'].value_counts()

2    435205
3    202389
1      9850
4      8683
5      5140
Name: Sets, dtype: int64

In [32]:
new_df = new_df[~new_df['Sets'].isin([1])]

In [33]:
new_df['Score'].unique()

array(['76,  64', '63 75 63', '63 63', ..., '16 63 61 46 61',
       '46 62 60 75', '63 06 10-7'], dtype=object)

In [34]:
new_df[['Start_Date', 'End_Date']] = new_df['Date'].str.split(' - ', expand=True) #Criação de 2 novas colunas

In [35]:
new_df['Start_Date'] = pd.to_datetime(new_df['Start_Date'], format='%Y.%m.%d')
new_df['End_Date'] = pd.to_datetime(new_df['End_Date'], format='%Y.%m.%d')

In [36]:
rafael_games = new_df[(new_df['PlayerName'] == 'Rafael Nadal') | (new_df['Oponent'] == 'Rafael Nadal')]

filtered_games = rafael_games[rafael_games['Start_Date'] < '2017-10-09']

filtered_games = filtered_games.sort_values('Date', ascending=False)
filtered_games.head(5)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,Score_Sets,Sets,Start_Date,End_Date
55865,Rafael Nadal,Beijing,"Beijing, China",2017.10.02 - 2017.10.08,Hard,Finals,19,Nick Kyrgios,W,62 61,62-61,2,2017-10-02,2017-10-08
55866,Rafael Nadal,Beijing,"Beijing, China",2017.10.02 - 2017.10.08,Hard,Quarter-Finals,17,John Isner,W,64 76,64-76,2,2017-10-02,2017-10-08
55867,Rafael Nadal,Beijing,"Beijing, China",2017.10.02 - 2017.10.08,Hard,Semi-Finals,8,Grigor Dimitrov,W,63 46 61,63-46-61,3,2017-10-02,2017-10-08
55868,Rafael Nadal,Beijing,"Beijing, China",2017.10.02 - 2017.10.08,Hard,Round of 16,42,Karen Khachanov,W,63 63,63-63,2,2017-10-02,2017-10-08
55869,Rafael Nadal,Beijing,"Beijing, China",2017.10.02 - 2017.10.08,Hard,Round of 32,23,Lucas Pouille,W,"46 76, 75",46-76-75,3,2017-10-02,2017-10-08


In [37]:
print(filtered_games)

              PlayerName Tournament         Location                     Date  \
55865       Rafael Nadal    Beijing   Beijing, China  2017.10.02 - 2017.10.08   
55866       Rafael Nadal    Beijing   Beijing, China  2017.10.02 - 2017.10.08   
55867       Rafael Nadal    Beijing   Beijing, China  2017.10.02 - 2017.10.08   
55868       Rafael Nadal    Beijing   Beijing, China  2017.10.02 - 2017.10.08   
55869       Rafael Nadal    Beijing   Beijing, China  2017.10.02 - 2017.10.08   
...                  ...        ...              ...                      ...   
56742       Rafael Nadal   Mallorca  Mallorca, Spain  2002.04.29 - 2002.05.05   
338531    Olivier Rochus   Mallorca  Mallorca, Spain  2002.04.29 - 2002.05.05   
56736       Rafael Nadal    Seville   Seville, Spain  2001.09.17 - 2001.09.23   
388015   Stefano Galvani    Seville   Seville, Spain  2001.09.17 - 2001.09.23   
544731  Guillermo Platel  Spain F10    Madrid, Spain  2001.09.10 - 2001.09.16   

       Ground       GameRou

In [38]:
print(filtered_games["Date"].unique())

['2017.10.02 - 2017.10.08' '2017.09.18 - 2017.09.24'
 '2017.08.28 - 2017.09.10' '2017.08.14 - 2017.08.20'
 '2017.08.07 - 2017.08.13' '2017.07.03 - 2017.07.16'
 '2017.05.29 - 2017.06.11' '2017.05.15 - 2017.05.21'
 '2017.05.08 - 2017.05.14' '2017.04.24 - 2017.04.30'
 '2017.04.17 - 2017.04.23' '2017.03.20 - 2017.04.02'
 '2017.03.06 - 2017.03.19' '2017.02.27 - 2017.03.04'
 '2017.01.16 - 2017.01.29' '2017.01.02 - 2017.01.08'
 '2016.10.10 - 2016.10.16' '2016.10.03 - 2016.10.09'
 '2016.08.29 - 2016.09.11' '2016.08.15 - 2016.08.21'
 '2016.08.08 - 2016.08.14' '2016.05.23 - 2016.06.05'
 '2016.05.09 - 2016.05.15' '2016.05.02 - 2016.05.08'
 '2016.04.18 - 2016.04.24' '2016.04.11 - 2016.04.17'
 '2016.03.21 - 2016.04.03' '2016.03.07 - 2016.03.20'
 '2016.02.15 - 2016.02.21' '2016.02.08 - 2016.02.14'
 '2016.01.18 - 2016.01.31' '2016.01.04 - 2016.01.10'
 '2015.11.16 - 2015.11.22' '2015.11.02 - 2015.11.08'
 '2015.10.26 - 2015.11.01' '2015.10.12 - 2015.10.18'
 '2015.10.05 - 2015.10.11' '2015.09.14 - 2015.

In [39]:
new_df['Year'] = new_df['Start_Date'].dt.year

In [40]:
new_df[((new_df['PlayerName'] == 'Rafael Nadal') | (new_df['Oponent'] == 'Rafael Nadal')) & (new_df['Year'] == 2017)]

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,Score_Sets,Sets,Start_Date,End_Date,Year
74,Rafael Nadal,ATP Masters 1000 Shanghai,"Shanghai, China",2017.10.09 - 2017.10.15,Hard,Round of 32,56,Jared Donaldson,W,62 61,62-61,2,2017-10-09,2017-10-15,2017
75,Rafael Nadal,Australian Open,"Melbourne, Australia",2017.01.16 - 2017.01.29,Hard,Round of 64,36,Marcos Baghdatis,W,63 61 63,63-61-63,3,2017-01-16,2017-01-29,2017
55860,Rafael Nadal,ATP Masters 1000 Paris,"Paris, France",2017.10.30 - 2017.11.05,Hard,Round of 16,36,Pablo Cuevas,W,"63 67, 63",63-67-63,3,2017-10-30,2017-11-05,2017
55861,Rafael Nadal,ATP Masters 1000 Paris,"Paris, France",2017.10.30 - 2017.11.05,Hard,Round of 32,55,Hyeon Chung,W,75 63,75-63,2,2017-10-30,2017-11-05,2017
55862,Rafael Nadal,ATP Masters 1000 Shanghai,"Shanghai, China",2017.10.09 - 2017.10.15,Hard,Quarter-Finals,9,Grigor Dimitrov,W,"64 67, 63",64-67-63,3,2017-10-09,2017-10-15,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77515,David Goffin,Nitto ATP Finals,"London, Great Britain",2017.11.13 - 2017.11.19,Hard,Round Robin,1,Rafael Nadal,W,"76, 67, 64",76-67-64,3,2017-11-13,2017-11-19,2017
87184,Nick Kyrgios,ATP Masters 1000 Cincinnati,"Cincinnati, OH, U.S.A.",2017.08.14 - 2017.08.20,Hard,Quarter-Finals,2,Rafael Nadal,W,62 75,62-75,2,2017-08-14,2017-08-20,2017
89871,Sam Querrey,Acapulco,"Acapulco, Mexico",2017.02.27 - 2017.03.04,Hard,Finals,6,Rafael Nadal,W,63 76,63-76,2,2017-02-27,2017-03-04,2017
109261,Milos Raonic,Brisbane,"Brisbane, Australia",2017.01.02 - 2017.01.08,Hard,Quarter-Finals,9,Rafael Nadal,W,46 63 64,46-63-64,3,2017-01-02,2017-01-08,2017


In [41]:
len(new_df)

651417

In [42]:
sorted_games = new_df.sort_values('Start_Date', ascending=False)
grouped_games = sorted_games.groupby(['PlayerName', 'Oponent'])

recent_perf_median = grouped_games['Sets'].rolling(window=5, min_periods=1).median().reset_index(level=0, drop=True)
recent_perf_median = recent_perf_median.groupby(level=0).apply(lambda x: np.clip(round(x.median()), 2, 3))

default_recent_perf = dict(zip(new_df['PlayerName'].unique(), new_df['Sets']))

new_df['PlayerName_RecentPerformance'] = new_df['PlayerName'].map(recent_perf_median).fillna(default_recent_perf).values
new_df['Oponent_RecentPerformance'] = new_df['Oponent'].map(recent_perf_median).fillna(default_recent_perf).values

In [43]:
new_df['PlayerName_RecentPerformance'].value_counts()

2.0    624597
3.0     26406
Name: PlayerName_RecentPerformance, dtype: int64

In [44]:
new_df['Oponent_RecentPerformance'].value_counts()

2.0    635832
3.0     15579
Name: Oponent_RecentPerformance, dtype: int64

In [45]:
sorted_games = new_df.sort_values('Start_Date', ascending=False)

grouped_games = sorted_games.groupby(['PlayerName', 'Oponent'])

recent_perf_mean = grouped_games['Sets'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
recent_perf_mean = recent_perf_mean.groupby(level=0).apply(lambda x: np.clip(round(x.mean()), 2, 3))

default_recent_perf = dict(zip(new_df['PlayerName'].unique(), new_df['Sets']))

new_df['PlayerName_RecentPerformance2'] = new_df['PlayerName'].map(recent_perf_mean).fillna(default_recent_perf).values
new_df['Oponent_RecentPerformance2'] = new_df['Oponent'].map(recent_perf_mean).fillna(default_recent_perf).values

In [46]:
new_df['PlayerName_RecentPerformance2'].value_counts()

2.0    505891
3.0    145112
Name: PlayerName_RecentPerformance2, dtype: int64

In [47]:
new_df['Oponent_RecentPerformance2'].value_counts()

2.0    542884
3.0    108527
Name: Oponent_RecentPerformance2, dtype: int64

In [48]:
new_df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,Score_Sets,Sets,Start_Date,End_Date,Year,PlayerName_RecentPerformance,Oponent_RecentPerformance,PlayerName_RecentPerformance2,Oponent_RecentPerformance2
0,Novak Djokovic,ATP Masters 1000 Canada,"Toronto, Canada",2016.07.25 - 2016.07.31,Hard,Quarter-Finals,8,Tomas Berdych,W,"76, 64",76-64,2,2016-07-25,2016-07-31,2016,3.0,2.0,3.0,3.0
1,Novak Djokovic,Roland Garros,"Paris, France",2016.05.23 - 2016.06.05,Clay,Quarter-Finals,8,Tomas Berdych,W,63 75 63,63-75-63,3,2016-05-23,2016-06-05,2016,3.0,2.0,3.0,3.0
2,Novak Djokovic,ATP Masters 1000 Miami,"Miami, FL, U.S.A.",2016.03.21 - 2016.04.03,Hard,Quarter-Finals,7,Tomas Berdych,W,63 63,63-63,2,2016-03-21,2016-04-03,2016,3.0,2.0,3.0,3.0
3,Novak Djokovic,ATP Finals,"London, Great Britain",2015.11.16 - 2015.11.22,Hard,Round Robin,6,Tomas Berdych,W,63 75,63-75,2,2015-11-16,2015-11-22,2015,3.0,2.0,3.0,3.0
4,Novak Djokovic,Doha,"Doha, Qatar",2016.01.04 - 2016.01.10,Hard,Semi-Finals,6,Tomas Berdych,W,63 76,63-76,2,2016-01-04,2016-01-10,2016,3.0,2.0,3.0,3.0
5,Novak Djokovic,ATP Masters 1000 Paris,"Paris, France",2015.11.02 - 2015.11.08,Hard,Quarter-Finals,5,Tomas Berdych,W,"76, 76",76-76,2,2015-11-02,2015-11-08,2015,3.0,2.0,3.0,3.0


In [49]:
new_df['PlayerName_RecentPerformance'].value_counts()

2.0    624597
3.0     26406
Name: PlayerName_RecentPerformance, dtype: int64

In [50]:
new_df['Oponent_RecentPerformance'].value_counts()

2.0    635832
3.0     15579
Name: Oponent_RecentPerformance, dtype: int64

In [51]:
new_df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,Score_Sets,Sets,Start_Date,End_Date,Year,PlayerName_RecentPerformance,Oponent_RecentPerformance,PlayerName_RecentPerformance2,Oponent_RecentPerformance2
0,Novak Djokovic,ATP Masters 1000 Canada,"Toronto, Canada",2016.07.25 - 2016.07.31,Hard,Quarter-Finals,8,Tomas Berdych,W,"76, 64",76-64,2,2016-07-25,2016-07-31,2016,3.0,2.0,3.0,3.0
1,Novak Djokovic,Roland Garros,"Paris, France",2016.05.23 - 2016.06.05,Clay,Quarter-Finals,8,Tomas Berdych,W,63 75 63,63-75-63,3,2016-05-23,2016-06-05,2016,3.0,2.0,3.0,3.0
2,Novak Djokovic,ATP Masters 1000 Miami,"Miami, FL, U.S.A.",2016.03.21 - 2016.04.03,Hard,Quarter-Finals,7,Tomas Berdych,W,63 63,63-63,2,2016-03-21,2016-04-03,2016,3.0,2.0,3.0,3.0
3,Novak Djokovic,ATP Finals,"London, Great Britain",2015.11.16 - 2015.11.22,Hard,Round Robin,6,Tomas Berdych,W,63 75,63-75,2,2015-11-16,2015-11-22,2015,3.0,2.0,3.0,3.0
4,Novak Djokovic,Doha,"Doha, Qatar",2016.01.04 - 2016.01.10,Hard,Semi-Finals,6,Tomas Berdych,W,63 76,63-76,2,2016-01-04,2016-01-10,2016,3.0,2.0,3.0,3.0
5,Novak Djokovic,ATP Masters 1000 Paris,"Paris, France",2015.11.02 - 2015.11.08,Hard,Quarter-Finals,5,Tomas Berdych,W,"76, 76",76-76,2,2015-11-02,2015-11-08,2015,3.0,2.0,3.0,3.0


In [52]:
sorted_df = new_df.sort_values('Start_Date')

sorted_df['PlayerName_LastTimePlayed'] = sorted_df.groupby('PlayerName')['Start_Date'].diff().dt.days
sorted_df['Oponent_LastTimePlayed'] = sorted_df.groupby('Oponent')['Start_Date'].diff().dt.days

mask = (sorted_df['Tournament'] == sorted_df.groupby('PlayerName')['Tournament'].shift(-1)) & (sorted_df['Start_Date'] == sorted_df.groupby('PlayerName')['Start_Date'].shift(-1))
sorted_df.loc[mask, 'PlayerName_LastTimePlayed'] = 1

mask = (sorted_df['Tournament'] == sorted_df.groupby('Oponent')['Tournament'].shift(-1)) & (sorted_df['Start_Date'] == sorted_df.groupby('Oponent')['Start_Date'].shift(-1))
sorted_df.loc[mask, 'Oponent_LastTimePlayed'] = 1

new_df['PlayerName_LastTimePlayed'] = sorted_df['PlayerName_LastTimePlayed']
new_df['Oponent_LastTimePlayed'] = sorted_df['Oponent_LastTimePlayed']

In [53]:
new_df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,...,Sets,Start_Date,End_Date,Year,PlayerName_RecentPerformance,Oponent_RecentPerformance,PlayerName_RecentPerformance2,Oponent_RecentPerformance2,PlayerName_LastTimePlayed,Oponent_LastTimePlayed
0,Novak Djokovic,ATP Masters 1000 Canada,"Toronto, Canada",2016.07.25 - 2016.07.31,Hard,Quarter-Finals,8,Tomas Berdych,W,"76, 64",...,2,2016-07-25,2016-07-31,2016,3.0,2.0,3.0,3.0,0.0,28.0
1,Novak Djokovic,Roland Garros,"Paris, France",2016.05.23 - 2016.06.05,Clay,Quarter-Finals,8,Tomas Berdych,W,63 75 63,...,3,2016-05-23,2016-06-05,2016,3.0,2.0,3.0,3.0,0.0,14.0
2,Novak Djokovic,ATP Masters 1000 Miami,"Miami, FL, U.S.A.",2016.03.21 - 2016.04.03,Hard,Quarter-Finals,7,Tomas Berdych,W,63 63,...,2,2016-03-21,2016-04-03,2016,3.0,2.0,3.0,3.0,1.0,14.0
3,Novak Djokovic,ATP Finals,"London, Great Britain",2015.11.16 - 2015.11.22,Hard,Round Robin,6,Tomas Berdych,W,63 75,...,2,2015-11-16,2015-11-22,2015,3.0,2.0,3.0,3.0,1.0,1.0
4,Novak Djokovic,Doha,"Doha, Qatar",2016.01.04 - 2016.01.10,Hard,Semi-Finals,6,Tomas Berdych,W,63 76,...,2,2016-01-04,2016-01-10,2016,3.0,2.0,3.0,3.0,0.0,49.0
5,Novak Djokovic,ATP Masters 1000 Paris,"Paris, France",2015.11.02 - 2015.11.08,Hard,Quarter-Finals,5,Tomas Berdych,W,"76, 76",...,2,2015-11-02,2015-11-08,2015,3.0,2.0,3.0,3.0,0.0,21.0


In [54]:
new_df[new_df['PlayerName'] == 'Rafael Nadal']

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,...,Sets,Start_Date,End_Date,Year,PlayerName_RecentPerformance,Oponent_RecentPerformance,PlayerName_RecentPerformance2,Oponent_RecentPerformance2,PlayerName_LastTimePlayed,Oponent_LastTimePlayed
70,Rafael Nadal,Barcelona,"Barcelona, Spain",2019.04.22 - 2019.04.28,Clay,Round of 16,155,David Ferrer,W,63 63,...,2,2019-04-22,2019-04-28,2019,2.0,2.0,3.0,3.0,0.0,35.0
71,Rafael Nadal,ATP Masters 1000 Indian Wells,"Indian Wells, CA, U.S.A.",2019.03.04 - 2019.03.17,Hard,Round of 64,192,Jared Donaldson,W,61 61,...,2,2019-03-04,2019-03-17,2019,2.0,2.0,3.0,2.0,0.0,7.0
72,Rafael Nadal,Australian Open,"Melbourne, Australia",2019.01.14 - 2019.01.27,Hard,Round of 16,57,Tomas Berdych,W,60 61 76,...,3,2019-01-14,2019-01-27,2019,2.0,2.0,3.0,3.0,0.0,14.0
73,Rafael Nadal,US Open,"New York, NY, U.S.A.",2018.08.27 - 2018.09.09,Hard,Round of 128,148,David Ferrer,W,63 34 (RET),...,2,2018-08-27,2018-09-09,2018,2.0,2.0,3.0,3.0,1.0,14.0
74,Rafael Nadal,ATP Masters 1000 Shanghai,"Shanghai, China",2017.10.09 - 2017.10.15,Hard,Round of 32,56,Jared Donaldson,W,62 61,...,2,2017-10-09,2017-10-15,2017,2.0,2.0,3.0,2.0,0.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56768,Rafael Nadal,Stuttgart,"Stuttgart, Germany",2003.07.14 - 2003.07.20,Clay,Round of 64,317,Philipp Petzschner,W,75 61,...,2,2003-07-14,2003-07-20,2003,2.0,2.0,3.0,2.0,7.0,14.0
56782,Rafael Nadal,Spain F5,"Alicante, Spain",2002.07.08 - 2002.07.14,Clay,Semi-Finals,334,Oscar Hernandez,W,62 64,...,2,2002-07-08,2002-07-14,2002,2.0,2.0,3.0,2.0,0.0,28.0
56783,Rafael Nadal,ATP Masters 1000 Monte Carlo,"Monte Carlo, Monaco",2003.04.14 - 2003.04.20,Clay,Round of 64,49,Karol Kucera,W,61 62,...,2,2003-04-14,2003-04-20,2003,2.0,2.0,3.0,3.0,0.0,28.0
56784,Rafael Nadal,Hamburg,"Hamburg, Germany",2003.01.27 - 2003.02.02,Carpet,Semi-Finals,142,Andrei Stoliarov,W,64 76,...,2,2003-01-27,2003-02-02,2003,2.0,2.0,3.0,2.0,0.0,7.0


In [55]:
new_df.isna().sum()

PlayerName                           0
Tournament                           0
Location                             0
Date                                 0
Ground                              96
GameRound                            0
GameRank                             0
Oponent                              6
WL                                   0
Score                                0
Score_Sets                           0
Sets                                 0
Start_Date                           0
End_Date                          7007
Year                                 0
PlayerName_RecentPerformance       414
Oponent_RecentPerformance            6
PlayerName_RecentPerformance2      414
Oponent_RecentPerformance2           6
PlayerName_LastTimePlayed         7280
Oponent_LastTimePlayed           22349
dtype: int64

In [56]:
new_df['PlayerName'] = new_df['PlayerName'].astype(str)
new_df['Oponent'] = new_df['Oponent'].astype(str)

new_df['Player_Oponent'] = new_df[['PlayerName', 'Oponent']].apply(lambda x: '_'.join(sorted(x)), axis=1)

new_df.sort_values('Start_Date', inplace=True)
grouped = new_df.groupby('Start_Date')
mean_sets = grouped['Sets'].transform(lambda x: x.iloc[:-1].mean())
mean_sets[grouped.cumcount(ascending=False) == 0] = 0
mean_sets = mean_sets.round().clip(2, 3)
new_df['H2H'] = mean_sets

new_df.drop('Player_Oponent', axis=1, inplace=True)

In [57]:
new_df['PlayerName'] = new_df['PlayerName'].astype(str)
new_df['Oponent'] = new_df['Oponent'].astype(str)
new_df['Player_Oponent'] = new_df[['PlayerName', 'Oponent']].apply(lambda x: '_'.join(sorted(x)), axis=1)

new_df.sort_values('Start_Date', inplace=True)
grouped = new_df.groupby('Start_Date')
median_sets = grouped['Sets'].transform(lambda x: x.iloc[:-1].median())
median_sets[grouped.cumcount(ascending=False) == 0] = 0
median_sets = median_sets.round().clip(2, 3)
new_df['H2H_Median'] = median_sets
new_df.drop('Player_Oponent', axis=1, inplace=True)

In [58]:
new_df['H2H'].value_counts()

2.0    583155
3.0     68262
Name: H2H, dtype: int64

In [59]:
new_df['H2H_Median'].value_counts()

2.0    627561
3.0     23856
Name: H2H_Median, dtype: int64

In [60]:
new_df.head(6)

,PlayerName,Tournament,Location,Date,Ground,GameRound,GameRank,Oponent,WL,Score,...,End_Date,Year,PlayerName_RecentPerformance,Oponent_RecentPerformance,PlayerName_RecentPerformance2,Oponent_RecentPerformance2,PlayerName_LastTimePlayed,Oponent_LastTimePlayed,H2H,H2H_Median
44017,Peter Wright,US Championships,"Newport, Rhode Island, United States",1914.08.24 - 1914.09.01,Grass,Round of 64,-,E. Gould,W,60 75 36 60,...,1914-09-01,1914,2.0,3.0,2.0,3.0,NaN,NaN,2.0,2.0
37047,Adli El Shafei,Wimbledon,"London, Great Britain",1946.06.24 - 1946.07.05,Grass,Round of 128,-,George Godsell,W,36 57 63 64 63,...,1946-07-05,1946,2.0,3.0,2.0,3.0,1.0,NaN,3.0,3.0
37048,Adli El Shafei,Wimbledon,"London, Great Britain",1946.06.24 - 1946.07.05,Grass,Round of 64,-,R.W. Baker,W,63 62 62,...,1946-07-05,1946,2.0,3.0,2.0,3.0,0.0,NaN,2.0,2.0
37046,Adli El Shafei,French Championships,"Paris, France",1946.07.15 - 1946.07.28,Clay,Round of 64,-,Stanley Drobac,W,62 61 60,...,1946-07-28,1946,2.0,3.0,2.0,3.0,21.0,NaN,2.0,2.0
37044,Adli El Shafei,French Championships,"Paris, France",1948.05.17 - 1948.05.30,Clay,Round of 64,-,Subba Sawhney,W,62 86 61,...,1948-05-30,1948,2.0,3.0,2.0,3.0,1.0,NaN,3.0,3.0
37045,Adli El Shafei,French Championships,"Paris, France",1948.05.17 - 1948.05.30,Clay,Round of 128,-,Jean Hugron,W,61 62 61,...,1948-05-30,1948,2.0,3.0,2.0,3.0,0.0,NaN,2.0,2.0


In [61]:
new_df.to_csv('df_grande_final.csv', index=False)